In [2]:
# from comet_ml import Experiment

# experiment = Experiment(api_key="oda8KKpxlDgWmJG5KsYrrhmIV", project_name="consensusnet")

import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Dropout
from keras.callbacks import EarlyStopping, TensorBoard

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Using Sequence generator

In [2]:
import numpy as np
from keras.utils import Sequence


class PileupSequence(Sequence):
    """
    Class for creating batches while training.

    Instance of this class is provided in fit_generator() method for training.
    """

    def __init__(self, X_paths, classes, batch_size, num_classes):
        """
        :param X_paths: list of paths to data
        :type X_paths: list of str
        :param classes: labels for data
        :type classes: np.ndarray
        :param batch_size: size of training batch
        :type batch_size: int
        :param num_classes: number of labels classes
        :type num_classes: int
        """
        if not len(X_paths) == len(classes):
            raise ValueError('You must provide same number of Xs and ys! '
                             'Number of Xs given is {], and number of ys '
                             'given is {}'.format(len(X_paths), len(classes)))
        if batch_size < 1 or batch_size > len(X_paths):
            raise ValueError('Batch size must be positive number less than '
                             'number of Xs, but {} given.'.format(batch_size))

        self.X_paths, self.classes = X_paths, classes
        self.batch_size = batch_size
        self.num_classes = num_classes

    def __len__(self):
        return int(np.ceil(len(self.X_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.X_paths[
                  idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.classes[
                  idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([np.load(file_name) for file_name in batch_x]), \
            _to_categorical(batch_y, num_classes=self.num_classes)


def _to_categorical(y, num_classes=None):
    """
    Converts given labels to one-hot encoding (i.e. categorical).

    :param y:
    :type y: np.ndarray
    :param num_classes:
    :type num_classes: int
    :return: Labels one-hot encoded.
    :rtype: np.ndarray
    """
    if num_classes < 1:
        raise ValueError('Number of classes must be positive int, but {} '
                         'given.'.format(num_classes))
    if not len(y.shape) == 1:
        raise ValueError('y must be 1-D numpy array, but {} shape '
                         'given.'.format(y.shape))

    n = y.shape[0]

    if num_classes is None:
        num_classes = np.max(y)

    categorical = np.zeros((n, num_classes), dtype=np.uint8)
    categorical[np.arange(n), y] = 1
    return categorical

In [ ]:
import os

classes = np.load('./dataset-classes-y.npy')
X_paths = [os.path.join('./X/', xi) for xi in os.listdir('./X/')]
batch_size = 1
num_classes = 4

print('done!')

In [10]:
batch_size = 100
generator = PileupSequence(X_paths, classes, batch_size, num_classes)

example_shape = (21, 1, 6)
input_layer = Input(shape=example_shape)

conv_1 = Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(input_layer)

conv_2 = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(conv_1)

flatten = Flatten()(conv_2)
dense_1 = Dense(1042)(flatten)
dropout_1 = Dropout(0.25)(dense_1)
predictions = Dense(4, activation='softmax')(dropout_1)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

# batch_size = 10000
epochs = 3

# model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))
model.fit_generator(generator, epochs=epochs, use_multiprocessing=True, workers=12)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 21, 1, 6)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 21, 1, 16)         880       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 21, 1, 32)         4640      
_________________________________________________________________
flatten_8 (Flatten)          (None, 672)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1042)              701266    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1042)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 4172      
Total para

Process ForkPoolWorker-67:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-69:
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]


KeyboardInterrupt: 

  File "<ipython-input-2-f8c5d0676194>", line 44, in __getitem__
    return np.array([np.load(file_name) for file_name in batch_x]),             _to_categorical(batch_y, num_classes=self.num_classes)
Process ForkPoolWorker-66:
  File "<ipython-input-2-f8c5d0676194>", line 44, in <listcomp>
    return np.array([np.load(file_name) for file_name in batch_x]),             _to_categorical(batch_y, num_classes=self.num_classes)
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/numpy/lib/npyio.py", line 404, in load
    magic = fid.read(N)
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in 

  File "/usr/local/lib/python3.5/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
KeyboardInterrupt
  File "<ipython-input-2-f8c5d0676194>", line 44, in __getitem__
    return np.array([np.load(file_name) for file_name in batch_x]),             _to_categorical(batch_y, num_classes=self.num_classes)
Traceback (most recent call last):
  File "<ipython-input-2-f8c5d0676194>", line 44, in <listcomp>
    return np.array([np.load(file_name) for file_name in batch_x]),             _to_categorical(batch_y, num_classes=self.num_classes)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/numpy/lib/npyio.py", line 404, in load
    magic = fid.read(N)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker

## Read all in RAM

In [3]:
X_train = np.load('./dataset-n10-X-reshaped-train.npy')
X_validate = np.load('./dataset-n10-X-reshaped-validate.npy')
y_train = np.load('./dataset-n10-y-reshaped-train.npy')
y_validate = np.load('./dataset-n10-y-reshaped-validate.npy')

In [6]:
example_shape = X_train.shape[1:]
input_layer = Input(shape=example_shape)

conv_1 = Conv2D(filters=5, kernel_size=3, padding='same', activation='relu')(input_layer)

conv_2 = Conv2D(filters=5, kernel_size=3, padding='same', activation='relu')(conv_1)

flatten = Flatten()(conv_2)
dense_1 = Dense(10)(flatten)
dropout_1 = Dropout(0.25)(dense_1)
predictions = Dense(4, activation='softmax')(dropout_1)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

batch_size = 50000
epochs = 5

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21, 1, 6)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 1, 5)          275       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 1, 5)          230       
_________________________________________________________________
flatten_4 (Flatten)          (None, 105)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1060      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 44        
Total para

In [7]:
model.save('./model-slim.h5')

In [4]:
from keras.models import load_model

model = load_model('./model-slim.h5')

batch_size = 50000
epochs = 5

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Train on 15258174 samples, validate on 1695353 samples
Epoch 1/5
15258174/15258174 [==============================] - 72s 5us/step - loss: 0.0947 - acc: 0.9753 - val_loss: 0.0797 - val_acc: 0.9767
Epoch 2/5
15258174/15258174 [==============================] - 64s 4us/step - loss: 0.0893 - acc: 0.9760 - val_loss: 0.0743 - val_acc: 0.9775
Epoch 3/5
15258174/15258174 [==============================] - 64s 4us/step - loss: 0.0828 - acc: 0.9769 - val_loss: 0.0692 - val_acc: 0.9783
Epoch 4/5
15258174/15258174 [==============================] - 67s 4us/step - loss: 0.0766 - acc: 0.9776 - val_loss: 0.0648 - val_acc: 0.9789
Epoch 5/5
15258174/15258174 [==============================] - 62s 4us/step - loss: 0.0731 - acc: 0.9779 - val_loss: 0.0634 - val_acc: 0.9792


In [5]:
batch_size = 50000
epochs = 5

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Train on 15258174 samples, validate on 1695353 samples
Epoch 1/5
15258174/15258174 [==============================] - 61s 4us/step - loss: 0.0710 - acc: 0.9780 - val_loss: 0.0619 - val_acc: 0.9793
Epoch 2/5
15258174/15258174 [==============================] - 65s 4us/step - loss: 0.0697 - acc: 0.9782 - val_loss: 0.0611 - val_acc: 0.9796
Epoch 3/5
15258174/15258174 [==============================] - 65s 4us/step - loss: 0.0684 - acc: 0.9784 - val_loss: 0.0601 - val_acc: 0.9797
Epoch 4/5
15258174/15258174 [==============================] - 65s 4us/step - loss: 0.0672 - acc: 0.9786 - val_loss: 0.0594 - val_acc: 0.9798
Epoch 5/5
15258174/15258174 [==============================] - 64s 4us/step - loss: 0.0666 - acc: 0.9787 - val_loss: 0.0599 - val_acc: 0.9792


In [6]:
batch_size = 50000
epochs = 5

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Train on 15258174 samples, validate on 1695353 samples
Epoch 1/5
15258174/15258174 [==============================] - 60s 4us/step - loss: 0.0661 - acc: 0.9787 - val_loss: 0.0590 - val_acc: 0.9800
Epoch 2/5
15258174/15258174 [==============================] - 62s 4us/step - loss: 0.0656 - acc: 0.9788 - val_loss: 0.0582 - val_acc: 0.9796
Epoch 3/5
15258174/15258174 [==============================] - 64s 4us/step - loss: 0.0649 - acc: 0.9789 - val_loss: 0.0579 - val_acc: 0.9803
Epoch 4/5
15258174/15258174 [==============================] - 58s 4us/step - loss: 0.0642 - acc: 0.9789 - val_loss: 0.0576 - val_acc: 0.9794
Epoch 5/5
15258174/15258174 [==============================] - 61s 4us/step - loss: 0.0636 - acc: 0.9790 - val_loss: 0.0568 - val_acc: 0.9803


In [7]:
batch_size = 50000
epochs = 5

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Train on 15258174 samples, validate on 1695353 samples
Epoch 1/5
15258174/15258174 [==============================] - 57s 4us/step - loss: 0.0635 - acc: 0.9791 - val_loss: 0.0569 - val_acc: 0.9799
Epoch 2/5
15258174/15258174 [==============================] - 57s 4us/step - loss: 0.0632 - acc: 0.9792 - val_loss: 0.0567 - val_acc: 0.9805
Epoch 3/5
15258174/15258174 [==============================] - 56s 4us/step - loss: 0.0630 - acc: 0.9792 - val_loss: 0.0563 - val_acc: 0.9803
Epoch 4/5
15258174/15258174 [==============================] - 58s 4us/step - loss: 0.0629 - acc: 0.9792 - val_loss: 0.0581 - val_acc: 0.9803
Epoch 5/5
15258174/15258174 [==============================] - 57s 4us/step - loss: 0.0628 - acc: 0.9792 - val_loss: 0.0561 - val_acc: 0.9806


In [8]:
model.save('./model-slim-better.h5')